In [8]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm, trange
from IPython.display import clear_output
import os

os.chdir('/Users/vladimirdimitrov/NLP-case')

In [3]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(29564, 312, padding_idx=0)
    (position_embeddings): Embedding(512, 312)
    (token_type_embeddings): Embedding(2, 312)
    (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-2): 3 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=312, out_features=312, bias=True)
            (key): Linear(in_features=312, out_features=312, bias=True)
            (value): Linear(in_features=312, out_features=312, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=312, out_features=312, bias=True)
            (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
    

In [9]:
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings.cpu().numpy()

In [14]:
df = pd.read_csv('data/interim/hh_database_clear.csv', index_col=0)

In [17]:
df['Text'] = df['Опыт работы'] + '. ' + df['Последнее/нынешнее место работы'] + '. ' + df['Последняя/нынешняя должность'] + '. ' + df['Образование и ВУЗ'] + '. ' + str(df['Money']) + df['Rate']

In [18]:
list_text = df['Text'].to_list()

In [19]:
len(list_text)

44423

In [20]:
BATCH_SIZE = 1024

In [21]:
for i in trange(len(list_text) // BATCH_SIZE):
    if i == 0:
        embed = embed_bert_cls(list_text[i * BATCH_SIZE : (i+1) * BATCH_SIZE], model, tokenizer)
        dataframe = pd.DataFrame(embed)
        continue
    embed = embed_bert_cls(list_text[i * BATCH_SIZE : (i+1) * BATCH_SIZE], model, tokenizer)
    dataframe = pd.concat([dataframe, pd.DataFrame(embed).reset_index(drop=True)], ignore_index=True, axis=0)
    print(dataframe.shape)
clear_output()
last = list_text[(i+1) * BATCH_SIZE:]
embed = embed_bert_cls(last, model, tokenizer)
dataframe = pd.concat([dataframe, pd.DataFrame(embed).reset_index(drop=True)], ignore_index=True, axis=0)

  0%|          | 0/43 [00:00<?, ?it/s]

In [11]:
dataframe.to_csv('embed.csv')